# Agent Component: Digital Twin Agent

The `DigitalTwinAgent` is a type of **Perception Agent**. Its sole responsibility is to act as a digital counterpart to a physical object in the simulation. It reads the state of its associated physical object and publishes that state to the `MessageBus`.

This simulates the role of a sensor in a real-world system, providing a stream of data that other agents can subscribe and react to. This is a fundamental component for creating a decoupled, event-driven multi-agent system.

## Simulation Example

In the simulation below, we will:
1. Create a `Reservoir` as our physical object.
2. Create a `DigitalTwinAgent` and link it to the reservoir.
3. Create a simple `listener` function that subscribes to the agent's topic on the `MessageBus` and records all messages it receives.
4. In a loop, we will manually change the reservoir's volume and then call the agent's `run()` method.
5. Finally, we will inspect the messages captured by our listener to verify that the agent correctly published the state changes.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.core.interfaces import State, Parameters

# 1. Setup components
message_bus = MessageBus()
RESERVOIR_STATE_TOPIC = "state.reservoir.level"

initial_state = State(volume=25e6, water_level=15.0)
parameters = Parameters(surface_area=1.5e6)
reservoir = Reservoir(name="storage_reservoir", initial_state=initial_state, parameters=parameters)

twin_agent = DigitalTwinAgent(
    agent_id="twin_for_reservoir",
    simulated_object=reservoir,
    message_bus=message_bus,
    state_topic=RESERVOIR_STATE_TOPIC
)

# 2. Create a listener to spy on the message bus
captured_messages = []
def message_listener(message):
    captured_messages.append(message)

message_bus.subscribe(RESERVOIR_STATE_TOPIC, message_listener)

# 3. Simulation loop
num_steps = 5
for i in range(num_steps):
    print(f"--- Step {i+1} ---")
    # Manually change the reservoir's state
    new_volume = reservoir.get_state()['volume'] + (i * 1e5)
    reservoir.set_state({'volume': new_volume, 'water_level': new_volume / parameters['surface_area']})
    print(f"Manually set reservoir volume to: {reservoir.get_state()['volume']:.2f}")
    
    # Run the agent
    twin_agent.run(current_time=i)
    print(f"DigitalTwinAgent was run. A message should have been published.")

print("\n--- Simulation Finished ---")

## Results

Below are the messages that our listener captured from the message bus. As you can see, for each step in our simulation where we changed the reservoir's state and ran the agent, a corresponding message containing the new state was published to the correct topic.

In [ ]:
# Display the captured messages
if not captured_messages:
    print("No messages were captured.")
else:
    print(f"Captured {len(captured_messages)} messages from topic '{RESERVOIR_STATE_TOPIC}':\n")
    for i, msg in enumerate(captured_messages):
        print(f"Message {i+1}: {msg}")